In [1]:
import numpy as np
import pandas as pd
import os
from scipy import stats
import utils
import torch.optim as optim
import torch
from torch.utils.data.sampler import RandomSampler

import model.net as net
from dataloader import *
from train import train_and_evaluate

months = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

In [2]:
data_set = 'Irish_2010'
path = os.path.abspath(os.path.join(os.getcwd(), '../..'))

data = get_data(path, data_set)

100%|██████████| 918/918 [00:29<00:00, 31.53it/s]


In [3]:
month = 1
n_clusters = 2
method = 'hierarchical/euclidean'

path_cluster = os.path.join(path, 'result', data_set, 'clustering', 'point', method, f'n_clusters_{n_clusters}.csv')
clusters = pd.read_csv(path_cluster, header=None)
path_data = os.path.join(path, 'data', 'deepar')

series = data[:, month-1, :months[month-1]*24]

weather = get_weather(path, data_set, month)
week = get_dow(data_set, month)
day = get_hod(month)

num_covariates = 4
covariates = np.zeros((num_covariates, len(series[0])))
covariates[1] = stats.zscore(weather)
covariates[2] = stats.zscore(week)
covariates[3] = stats.zscore(day)
covariates = covariates.T

In [ ]:
for i in range(n_clusters):

    index = list(clusters[month-1] == i)
    sub_series = series[index]
    
    test_data = sub_series[:, -168*2:].T
    train_data = sub_series[:, :-168].T
    
    data_start = (train_data != 0).argmax(axis=0)
    total_time = sub_series.shape[1]
    num_series = sub_series.shape[0]
    
    window_size = 192
    stride_size = 24
    
    # prepare data
    cov = covariates.copy()
    train_x_input, train_v_input, train_label = prep_data(train_data, cov, data_start, window_size, stride_size, num_covariates, num_series, total_time)
    cov = covariates.copy()
    test_x_input, test_v_input, test_label = prep_data(test_data, cov, data_start, window_size, stride_size, num_covariates, num_series, total_time, train=False)
    
    # params
    json_path = os.path.join(path, 'forecasting', 'deepar', 'params24.json')
    params = utils.Params(json_path)
    
    params.num_class = np.sum(index)
    params.relative_metrics = False
    params.sampling = False
    params.one_step = True
    
    # use GPU if available
    cuda_exist = torch.cuda.is_available()
    # Set random seeds for reproducible experiments if necessary
    if cuda_exist:
        params.device = torch.device('cuda')
        # torch.cuda.manual_seed(240)
        model = net.Net(params).cuda()
    else:
        params.device = torch.device('cpu')
        # torch.manual_seed(230)
        model = net.Net(params)
    
    train_set = TrainDataset(train_x_input, train_label)
    test_set = TestDataset(test_x_input, test_v_input, test_label)
    sampler = WeightedSampler(train_v_input) # Use weighted sampler instead of random sampler
    train_loader = DataLoader(train_set, batch_size=params.batch_size, sampler=sampler, num_workers=8)
    test_loader = DataLoader(test_set, batch_size=params.predict_batch, sampler=RandomSampler(test_set), num_workers=8)
    
    optimizer = optim.Adam(model.parameters(), lr=params.learning_rate)
    loss_fn = net.loss_fn
    
    restore_file = None
    train_and_evaluate(model,
                       train_loader,
                       test_loader,
                       optimizer,
                       loss_fn,
                       params,
                       restore_file)
    break

Begin training
Epoch 1/1000
train_loss: 1.2481319904327393
train_loss: 0.5764206250508627
- Full test metrics: ND: 0.596; RMSE: 1.004; test_loss: 0.950
Epoch 2/1000
train_loss: 0.35280577341715497
train_loss: 0.21474790573120117
- Full test metrics: ND: 0.538; RMSE: 0.951; test_loss: 0.753
Epoch 3/1000
train_loss: 0.21759355068206787
train_loss: 0.14128215114275613
- Full test metrics: ND: 0.533; RMSE: 0.941; test_loss: 0.681
Epoch 4/1000
train_loss: 0.12473193804423015
train_loss: 0.0874929428100586
- Full test metrics: ND: 0.536; RMSE: 0.935; test_loss: 0.641
Epoch 5/1000
train_loss: 0.08409753441810608
train_loss: 0.0900423030058543
- Full test metrics: ND: 0.519; RMSE: 0.931; test_loss: 0.626
Epoch 6/1000
train_loss: -0.014974930634101232
train_loss: 0.04218562444051107
- Full test metrics: ND: 0.521; RMSE: 0.928; test_loss: 0.597
Epoch 7/1000
train_loss: -0.011313812186320623
train_loss: -0.03911385933558146
- Full test metrics: ND: 0.518; RMSE: 0.926; test_loss: 0.608
Epoch 8/100

train_loss: -0.3268589774767558
train_loss: -0.20221741994222006
- Full test metrics: ND: 0.494; RMSE: 0.914; test_loss: 0.649
Epoch 60/1000
train_loss: -0.2906075716018677
train_loss: -0.3714430332183838
- Full test metrics: ND: 0.493; RMSE: 0.914; test_loss: 0.682
Epoch 61/1000
train_loss: -0.2475046714146932
train_loss: -0.2649671832720439
- Full test metrics: ND: 0.493; RMSE: 0.913; test_loss: 0.673
Epoch 62/1000
train_loss: -0.3099132577578227
train_loss: -0.2722376187642415
- Full test metrics: ND: 0.496; RMSE: 0.913; test_loss: 0.668
Epoch 63/1000
train_loss: -0.26852885882059735
train_loss: -0.21689865986506143
- Full test metrics: ND: 0.495; RMSE: 0.914; test_loss: 0.660
Epoch 64/1000
train_loss: -0.33537499109903973
train_loss: -0.334644357363383
- Full test metrics: ND: 0.499; RMSE: 0.913; test_loss: 0.677
Epoch 65/1000
train_loss: -0.3843332926432292
train_loss: -0.30988738934199017
